In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import methods as m
import pickle
import warnings
import plotly as py
import os

warnings.filterwarnings('ignore')

In [ ]:
def get_list_videos(vid_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(vid_dir):
        f.extend(filenames)
    return f


def run_openpose(vid_dir,coord_location, openpose_location):
    os.chdir(openpose_location)
    for video in get_list_videos(vid_dir):
        os.system(r'bin\OpenPoseDemo.exe --video "{0}\{1}" --write_json "{2}\{1}"'.format(vid_dir, video, coord_location))

In [ ]:
def prepare_data_for_classification(vid_dir, coord_location, openpose_location):
    #run_openpose(vid_dir,coord_location, openpose_location)
    video_number = 1
    return_df = None
    for video in get_list_videos(vid_dir):
        print(video)
        image_h, image_w, fps = m.determine_video_meta_data(r'{}\{}'.format(vid_dir, video))

        #        people_per_file = m.get_openpose_output(r'{}\{}'.format(coord_location,video))
        period_person_division = m.get_period_person_division(
            m.get_openpose_output(r'{}\{}'.format(coord_location, video)), fps)
        person_period_division = m.get_person_period_division(period_person_division)

        mean_x_per_person = m.get_mean_x_per_person(person_period_division)

        normalized_moved_distance_per_person = m.normalize_moved_distance_per_person(mean_x_per_person)
        maximum_normalized_distance = max(normalized_moved_distance_per_person.values())

        movement_threshold = maximum_normalized_distance / 4

        moving_people = [key for key, value in normalized_moved_distance_per_person.items() if
                         value > movement_threshold]

        period_running_person_division_df = m.get_period_running_person_division_df(mean_x_per_person, moving_people)

        dbscan_subsets = m.get_dbscan_subsets(maximum_normalized_distance, period_running_person_division_df)
        max_dbscan_subset = dbscan_subsets[
            np.argmax([sum([len(person_period_division[person]) for person in subset]) for subset in dbscan_subsets])]

        running_person_identifiers = m.determine_running_person_identifiers(period_running_person_division_df,
                                                                            max_dbscan_subset,
                                                                            maximum_normalized_distance * 4,
                                                                            maximum_normalized_distance ** 2)

        running_fragments, turning_fragments, fragments = m.get_running_and_turning_fragments(
            running_person_identifiers,
            mean_x_per_person,
            period_running_person_division_df,
            moving_people,
            fps)
        
        coord_df = m.get_dataframe_from_coords(period_person_division,
                                               running_person_identifiers,
                                               running_fragments)

        rotation_angle = m.get_rotation_angle(coord_df)

        period_person_division = {
        period: {person: np.array([m.rotate((x, y), rotation_angle) + (z,) for x, y, z in coords])
                 for person, coords in period_dictionary.items()}
        for period, period_dictionary in period_person_division.items()}

        period_running_person_division, running_plottables, turning_plottables = \
            m.get_plottables(period_person_division, running_person_identifiers, running_fragments, turning_fragments)

        return_df = m.create_total_feature_df(coord_df, video_number, return_df,
                                              period_running_person_division, running_fragments, fragments, fps)
        video_number += 1
    return return_df



df = prepare_data_for_classification(r'C:\Users\jaspe\tf-openpose\clips',
                                     r'C:\Users\jaspe\tf-openpose\demo\openpose-1.2.1-win64-binaries\coordinates',
                                     r'C:\Users\jaspe\tf-openpose\demo\openpose-1.2.1-win64-binaries')

# prepare_data_for_classification(r'C:\Users\herbe\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\videos',
#              r'C:\Users\herbe\Dropbox\TUe\DS-E\2017-2018\JM0130 Data Entrepreneurship in Action II\4. 2D_pose_estimation\coordinates',
#             r'C:\Users\Herbert van Leeuwen\Desktop\openpose-1.2.1-win64-binaries\openpose-1.2.1-win64-binaries')


In [ ]:
df

In [ ]:
% matplotlib inline

In [ ]:
import seaborn as sns

features = list(set(df.columns[:-1]) - set(['video']))

f,ax = plt.subplots(nrows=len(features)//3+1, ncols=3, figsize=(18,5*len(features)//3))

for n, feature in enumerate(features):
    sns.regplot(data=df, x=feature, y='speed (km/h)', ax=ax[n//3][n%3])

In [ ]:
df.to_csv('df_for_colin.csv', sep=',')

In [ ]:
pickle.dump(df, open('df_for_colin.p', 'wb'))